## Using PyTorch Tensors 

A PyTorch Tensor is basically the same as a numpy multidimensional array: it does not know
anything about deep learning or computational graphs or gradients, and is just
a generic n-dimensional array to be used for arbitrary numeric computation.

The biggest difference between a numpy array and a PyTorch Tensor is that
a PyTorch Tensor can run on either CPU or GPU. To run operations on the GPU,
just cast the Tensor to a cuda datatype.

(This example is adapted from: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html )

### Basic tensor matrix operations

In [1]:
import torch

# Simple matrix
m1 = torch.ones(3, 4)
print('Matrix m1: \n', m1)
print(m1.shape,"\n")

# Another (random) matrix
m2 = torch.rand(3, 4) # fill 3x4 matrix with uniform random numbers in [0,1] interval
print('Matrix m2: \n', m2)
print(m2.shape,"\n")

# Transpose of a matrix
print('Matrix m2^T: \n', m2.t())
print(m2.t().shape,"\n")

# Matrix operations
m3 = m1*m2      # Not a matrix multiplication ! Here each value in matrix m1 are multiplied by value m2
print('Matrix m3: \n', m3)
print(m3.shape,"\n")

# Matrix multiplication using torch.mm
m4 = m1.mm(m2.t())
print('Matrix m4: \n', m4)
print(m4.shape,"\n")

# Matrix multiplication using torch.matmul
m5 = m1.matmul(m2.t())
print('Matrix m5: \n', m5)
print(m5.shape,"\n")

# For matrix-vector multiplication there is also the possibility to use torch.mv
vec = torch.tensor([1.0,2.0,3.0])
print(torch.matmul(m5, vec))

Matrix m1: 
 tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
torch.Size([3, 4]) 

Matrix m2: 
 tensor([[0.1157, 0.4705, 0.1604, 0.6519],
        [0.5901, 0.3330, 0.9829, 0.5998],
        [0.5548, 0.0507, 0.7762, 0.8113]])
torch.Size([3, 4]) 

Matrix m2^T: 
 tensor([[0.1157, 0.5901, 0.5548],
        [0.4705, 0.3330, 0.0507],
        [0.1604, 0.9829, 0.7762],
        [0.6519, 0.5998, 0.8113]])
torch.Size([4, 3]) 

Matrix m3: 
 tensor([[0.1157, 0.4705, 0.1604, 0.6519],
        [0.5901, 0.3330, 0.9829, 0.5998],
        [0.5548, 0.0507, 0.7762, 0.8113]])
torch.Size([3, 4]) 

Matrix m4: 
 tensor([[1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929]])
torch.Size([3, 3]) 

Matrix m5: 
 tensor([[1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929]])
torch.Size([3, 3]) 

tensor([12.9888, 12.9888, 12.9888])


### More operations (power, sum, clamp, ...)

(See even more basic operations here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)

In [2]:
mat = torch.rand(3, 4)
print(mat)

# Takes the power of each element in input
print(mat.pow(3))

# Returns the sum of all elements in the input tensor
print(mat.sum())

# Clamp all elements in input into the range [ min, max ] and return a resulting tensor
print(mat.clamp(0.3,0.7))

tensor([[0.3053, 0.3329, 0.1608, 0.3585],
        [0.2057, 0.0540, 0.3088, 0.3913],
        [0.1058, 0.3738, 0.0864, 0.2156]])
tensor([[0.0285, 0.0369, 0.0042, 0.0461],
        [0.0087, 0.0002, 0.0294, 0.0599],
        [0.0012, 0.0522, 0.0006, 0.0100]])
tensor(2.8989)
tensor([[0.3053, 0.3329, 0.3000, 0.3585],
        [0.3000, 0.3000, 0.3088, 0.3913],
        [0.3000, 0.3738, 0.3000, 0.3000]])


### Convertion to numpy array

In [3]:
import numpy as np

# convertion PyTorch -> numpy
a = torch.randn(5)
b = a.numpy()

# convertion numpy -> PyTorch
c = torch.from_numpy(b)

print(a)
print(b)
print(c)

tensor([-1.7202,  0.8410, -0.3200,  1.4264,  1.1526])
[-1.7201914   0.8410047  -0.32003662  1.42642     1.1526212 ]
tensor([-1.7202,  0.8410, -0.3200,  1.4264,  1.1526])


### Autograd

A PyTorch Tensor represents a node in a computational graph. If ``x`` is a
Tensor that has ``x.requires_grad=True`` then ``x.grad`` is another Tensor
holding the gradient of ``x`` with respect to some scalar value.

Example for simple regression (adapted from S. Sviridov lecture):

In [4]:
# Simple regression example
x = [1., 2., 3., 4., 5.]           # data
y = [10., 20., 30., 40., 50.]      # target values

# Gradients will be calculated w.r.t this tensors (has "requires_grad=True")
w = torch.tensor([1.],requires_grad=True)

# Number of loops on all sample
for epoch in range(5):
    
    # Loop on data events and target values
    for x_i, y_i in zip(x, y):
        
        # compute predicted target variable
        y_pred = x_i * w
                
        # compute Mean Squared Error (MSE)
        loss = (y_pred - y_i) ** 2
        
        # With PyTorch we can automatically compute the derivative of the loss 
        # w.r.t. the tensors that have requires_grad set to True (i.e. w and b).
        # compute gradients
        loss.backward()
                        
        print('\t x=%.1f y=%.1f, w=%.2f, dloss/dw=%.1f' % (x_i, y_i, w.data, w.grad.data))
                
        # make one step towards the local minima, with learning rate 0.01
        w.data -= 0.01 * w.grad.data
        
        # clear gradients after updating weights
        w.grad.data.zero_()
        
    print('Loss at epoch #%d: %.6f \n' % (epoch+1, loss.data[0]))

print('Final: w = %.4f' % (w.data))


	 x=1.0 y=10.0, w=1.00, dloss/dw=-18.0
	 x=2.0 y=20.0, w=1.18, dloss/dw=-70.6
	 x=3.0 y=30.0, w=1.89, dloss/dw=-146.1
	 x=4.0 y=40.0, w=3.35, dloss/dw=-212.9
	 x=5.0 y=50.0, w=5.48, dloss/dw=-226.2
Loss at epoch #1: 511.797760 

	 x=1.0 y=10.0, w=7.74, dloss/dw=-4.5
	 x=2.0 y=20.0, w=7.78, dloss/dw=-17.7
	 x=3.0 y=30.0, w=7.96, dloss/dw=-36.7
	 x=4.0 y=40.0, w=8.33, dloss/dw=-53.5
	 x=5.0 y=50.0, w=8.86, dloss/dw=-56.9
Loss at epoch #2: 32.337894 

	 x=1.0 y=10.0, w=9.43, dloss/dw=-1.1
	 x=2.0 y=20.0, w=9.44, dloss/dw=-4.5
	 x=3.0 y=30.0, w=9.49, dloss/dw=-9.2
	 x=4.0 y=40.0, w=9.58, dloss/dw=-13.5
	 x=5.0 y=50.0, w=9.71, dloss/dw=-14.3
Loss at epoch #3: 2.043265 

	 x=1.0 y=10.0, w=9.86, dloss/dw=-0.3
	 x=2.0 y=20.0, w=9.86, dloss/dw=-1.1
	 x=3.0 y=30.0, w=9.87, dloss/dw=-2.3
	 x=4.0 y=40.0, w=9.89, dloss/dw=-3.4
	 x=5.0 y=50.0, w=9.93, dloss/dw=-3.6
Loss at epoch #4: 0.129104 

	 x=1.0 y=10.0, w=9.96, dloss/dw=-0.1
	 x=2.0 y=20.0, w=9.96, dloss/dw=-0.3
	 x=3.0 y=30.0, w=9.97, dloss/d

### Exercice
Now we have the following data. How should be modified the above code ? How many epochs are needed for convergence ?


In [6]:
# Now we have the following data. How should be modified the above code 
x = [1., 2., 3., 4., 5., 6., 7., 8., 9., 10.]           # data
y = [15., 25., 35., 45., 55., 65., 75., 85., 95., 105.] # target values

# Gradients will be calculated w.r.t these two tensors (have "requires_grad=True")
w = torch.tensor([1.],requires_grad=True)
b = torch.tensor([0.],requires_grad=True)

for epoch in range(100):
    
    for x_i, y_i in zip(x, y):
        
        # compute predicted target variable
        y_pred = x_i * w + b
                
        # compute Mean Squared Error (MSE)
        loss = (y_pred - y_i) ** 2
        
        # With PyTorch we can automatically compute the derivative of the loss 
        # w.r.t. the tensors that have requires_grad set to True (i.e. w and b).
        # compute gradients
        loss.backward()
                        
        print('\t x=%.1f y=%.1f, w=%.2f, b=%.2f, dloss/dw=%.1f, dloss/db=%.1f' % (x_i, y_i, w.data, b.data, w.grad.data, b.grad.data))
                
        # make one step towards the local minima, with learning rate 0.01
        w.data -= 0.01 * w.grad.data
        b.data -= 0.01 * b.grad.data
        
        # clear gradients after updating weights
        w.grad.data.zero_()
        b.grad.data.zero_()
        
    print('Loss at epoch #%d: %.6f \n' % (epoch+1, loss.data[0]))

print('Final: w = %.4f, b = %4f' % (w.data, b.data))


	 x=1.0 y=15.0, w=1.00, b=0.00, dloss/dw=-28.0, dloss/db=-28.0
	 x=2.0 y=25.0, w=1.28, b=0.28, dloss/dw=-88.6, dloss/db=-44.3
	 x=3.0 y=35.0, w=2.17, b=0.72, dloss/dw=-166.7, dloss/db=-55.6
	 x=4.0 y=45.0, w=3.83, b=1.28, dloss/dw=-227.1, dloss/db=-56.8
	 x=5.0 y=55.0, w=6.10, b=1.85, dloss/dw=-226.3, dloss/db=-45.3
	 x=6.0 y=65.0, w=8.37, b=2.30, dloss/dw=-150.0, dloss/db=-25.0
	 x=7.0 y=75.0, w=9.87, b=2.55, dloss/dw=-47.3, dloss/db=-6.8
	 x=8.0 y=85.0, w=10.34, b=2.62, dloss/dw=5.4, dloss/db=0.7
	 x=9.0 y=95.0, w=10.29, b=2.61, dloss/dw=3.3, dloss/db=0.4
	 x=10.0 y=105.0, w=10.25, b=2.61, dloss/dw=2.7, dloss/db=0.3
Loss at epoch #1: 0.018289 

	 x=1.0 y=15.0, w=10.23, b=2.60, dloss/dw=-4.3, dloss/db=-4.3
	 x=2.0 y=25.0, w=10.27, b=2.65, dloss/dw=-7.3, dloss/db=-3.6
	 x=3.0 y=35.0, w=10.34, b=2.68, dloss/dw=-7.7, dloss/db=-2.6
	 x=4.0 y=45.0, w=10.42, b=2.71, dloss/dw=-4.9, dloss/db=-1.2
	 x=5.0 y=55.0, w=10.47, b=2.72, dloss/dw=0.6, dloss/db=0.1
	 x=6.0 y=65.0, w=10.46, b=2.72, dlos

	 x=5.0 y=55.0, w=10.03, b=4.87, dloss/dw=0.0, dloss/db=0.0
	 x=6.0 y=65.0, w=10.03, b=4.87, dloss/dw=0.3, dloss/db=0.1
	 x=7.0 y=75.0, w=10.02, b=4.87, dloss/dw=0.4, dloss/db=0.1
	 x=8.0 y=85.0, w=10.02, b=4.87, dloss/dw=0.3, dloss/db=0.0
	 x=9.0 y=95.0, w=10.02, b=4.87, dloss/dw=0.2, dloss/db=0.0
	 x=10.0 y=105.0, w=10.01, b=4.87, dloss/dw=0.1, dloss/db=0.0
Loss at epoch #83: 0.000055 

	 x=1.0 y=15.0, w=10.01, b=4.87, dloss/dw=-0.2, dloss/db=-0.2
	 x=2.0 y=25.0, w=10.01, b=4.87, dloss/dw=-0.4, dloss/db=-0.2
	 x=3.0 y=35.0, w=10.02, b=4.87, dloss/dw=-0.4, dloss/db=-0.1
	 x=4.0 y=45.0, w=10.02, b=4.87, dloss/dw=-0.3, dloss/db=-0.1
	 x=5.0 y=55.0, w=10.03, b=4.87, dloss/dw=0.0, dloss/db=0.0
	 x=6.0 y=65.0, w=10.03, b=4.87, dloss/dw=0.3, dloss/db=0.1
	 x=7.0 y=75.0, w=10.02, b=4.87, dloss/dw=0.4, dloss/db=0.1
	 x=8.0 y=85.0, w=10.02, b=4.87, dloss/dw=0.3, dloss/db=0.0
	 x=9.0 y=95.0, w=10.02, b=4.87, dloss/dw=0.2, dloss/db=0.0
	 x=10.0 y=105.0, w=10.01, b=4.87, dloss/dw=0.1, dloss/db=0.


Optional: NN with tensor and autograd
-------------------------------

We consider a fully-connected ReLU network with one hidden layer and no biases, trained to predict y from x using Euclidean error.

The model that we want to build has the following structure:
$$\hat{y}(x) = \text{relu}(x.w_1).w_2,$$
where $x$ and $y$ are the input and output features (of dimension 1000 and 10, respectively). Here the relu activation function is used and $w_1$ and $w_2$ are weight matrices.

This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients.


In [ ]:
%matplotlib inline

import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # alternativelt you can use the built-in pytorch relu function
    # y_pred = torch.relu(x.mm(w1)).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

### Exercice
Add one more hidden layer with 100 neurons. Use successively sigmoid (torch.sigmoid) and relu (torch.relu) activation functions.

Note: you might need to increase the numbers of epochs (this solution is much less effective than the previous one !)


In [30]:
import numpy as np

%matplotlib inline

import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 64, 1000, 100, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H1, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H1, H2, device=device, dtype=dtype, requires_grad=True)
w3 = torch.randn(H2, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(10000):
    
    # y_pred with sigmoid activation function
    y_pred = torch.relu(torch.sigmoid(x.mm(w1)).mm(w2)).mm(w3)
    
    #y_pred = x.mm(w1).clamp(min=0).mm(w2).clamp(min=0).mm(w3)
        
    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w3 -= learning_rate * w3.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()
        w3.grad.zero_()

0 1179604.5
1 868468.5625
2 696925.25
3 592580.625
4 524517.3125
5 477580.28125
6 442786.125
7 415417.5625
8 392899.84375
9 373899.15625
10 357373.09375
11 343139.96875
12 330850.90625
13 319962.21875
14 310181.3125
15 301274.875
16 293071.46875
17 285374.5
18 278217.625
19 271477.71875
20 265055.59375
21 258905.28125
22 253058.515625
23 247456.53125
24 242098.4375
25 236934.171875
26 231957.921875
27 227161.890625
28 222582.4375
29 218207.28125
30 213994.921875
31 209888.21875
32 205889.609375
33 201961.0625
34 198125.765625
35 194370.875
36 190744.96875
37 187161.5
38 183700.53125
39 180380.421875
40 177165.890625
41 174000.265625
42 170909.734375
43 167875.71875
44 164900.875
45 162009.09375
46 159214.390625
47 156504.984375
48 153889.796875
49 151356.078125
50 148930.359375
51 146606.46875
52 144348.3125
53 142159.59375
54 140042.03125
55 137979.921875
56 135975.15625
57 133996.59375
58 132045.734375
59 130139.1484375
60 128262.7890625
61 126429.4765625
62 124632.796875
63 122863.0

499 8372.4169921875
500 8342.6298828125
501 8313.0146484375
502 8283.6298828125
503 8254.1640625
504 8224.9599609375
505 8196.0908203125
506 8167.263671875
507 8138.42724609375
508 8109.91845703125
509 8081.501953125
510 8053.04736328125
511 8024.99658203125
512 7996.84912109375
513 7968.98583984375
514 7941.08203125
515 7913.576171875
516 7886.0849609375
517 7858.36376953125
518 7831.4326171875
519 7804.017578125
520 7777.111328125
521 7750.404296875
522 7723.27978515625
523 7696.623046875
524 7669.93017578125
525 7643.3125
526 7616.98486328125
527 7590.44580078125
528 7564.47900390625
529 7538.51708984375
530 7512.38818359375
531 7486.58984375
532 7461.08349609375
533 7435.29443359375
534 7410.15625
535 7385.01123046875
536 7359.56396484375
537 7334.77734375
538 7309.73193359375
539 7285.12841796875
540 7260.60205078125
541 7235.77978515625
542 7211.630859375
543 7187.16015625
544 7162.9091796875
545 7139.14599609375
546 7114.8037109375
547 7090.81494140625
548 7067.37841796875
549 7

986 1995.93798828125
987 1990.94921875
988 1986.113037109375
989 1981.344482421875
990 1976.39404296875
991 1971.6051025390625
992 1966.89013671875
993 1961.9984130859375
994 1957.2294921875
995 1952.577880859375
996 1947.7669677734375
997 1943.0308837890625
998 1938.3643798828125
999 1933.662841796875
1000 1928.947509765625
1001 1924.3109130859375
1002 1919.6588134765625
1003 1915.0062255859375
1004 1910.39697265625
1005 1905.8446044921875
1006 1901.185791015625
1007 1896.6170654296875
1008 1892.139404296875
1009 1887.550537109375
1010 1882.9356689453125
1011 1878.4775390625
1012 1874.011474609375
1013 1869.4068603515625
1014 1864.9345703125
1015 1860.508056640625
1016 1855.9384765625
1017 1851.4925537109375
1018 1847.0765380859375
1019 1842.5904541015625
1020 1838.1798095703125
1021 1833.8402099609375
1022 1829.427490234375
1023 1824.9986572265625
1024 1820.6846923828125
1025 1816.2718505859375
1026 1812.0079345703125
1027 1807.657470703125
1028 1803.3048095703125
1029 1799.060302734

1458 734.6864013671875
1459 733.3159790039062
1460 731.9485473632812
1461 730.59130859375
1462 729.2036743164062
1463 727.89501953125
1464 726.4913940429688
1465 725.1429443359375
1466 723.7950439453125
1467 722.4912109375
1468 721.1082763671875
1469 719.7607421875
1470 718.4508056640625
1471 717.1024780273438
1472 715.766845703125
1473 714.4319458007812
1474 713.1456298828125
1475 711.798583984375
1476 710.4876708984375
1477 709.1569213867188
1478 707.8533325195312
1479 706.536865234375
1480 705.2332153320312
1481 703.9149169921875
1482 702.59130859375
1483 701.2750244140625
1484 699.9557495117188
1485 698.6556396484375
1486 697.3757934570312
1487 696.0677490234375
1488 694.7366943359375
1489 693.5140380859375
1490 692.1900634765625
1491 690.9382934570312
1492 689.5879516601562
1493 688.4011840820312
1494 687.0679321289062
1495 685.7921142578125
1496 684.5494384765625
1497 683.2601928710938
1498 682.052001953125
1499 680.7426147460938
1500 679.482177734375
1501 678.2407836914062
1502 

1821 386.8207702636719
1822 386.1727600097656
1823 385.5055236816406
1824 384.8625183105469
1825 384.2398376464844
1826 383.57379150390625
1827 382.9405822753906
1828 382.3013916015625
1829 381.6544189453125
1830 381.0196228027344
1831 380.3813781738281
1832 379.7278747558594
1833 379.1108093261719
1834 378.4823303222656
1835 377.8381042480469
1836 377.2090148925781
1837 376.56890869140625
1838 375.9527893066406
1839 375.3274230957031
1840 374.6815490722656
1841 374.0712890625
1842 373.4711608886719
1843 372.8155517578125
1844 372.19927978515625
1845 371.5690002441406
1846 370.9700012207031
1847 370.3448486328125
1848 369.7196044921875
1849 369.1163024902344
1850 368.50933837890625
1851 367.8828125
1852 367.26824951171875
1853 366.6698303222656
1854 366.0450134277344
1855 365.4556884765625
1856 364.8363342285156
1857 364.22161865234375
1858 363.62872314453125
1859 363.0318908691406
1860 362.41455078125
1861 361.8207702636719
1862 361.2138977050781
1863 360.626220703125
1864 360.0377197

...

8176 0.2193627506494522
8177 0.21913909912109375
8178 0.218928724527359
8179 0.21871623396873474
8180 0.21849828958511353
8181 0.2182803452014923
8182 0.2180626541376114
8183 0.21784844994544983
8184 0.2176332324743271
8185 0.21742458641529083
8186 0.2172187715768814
8187 0.21698780357837677
8188 0.21677687764167786
8189 0.21656814217567444
8190 0.2163560539484024
8191 0.21613620221614838
8192 0.2159225046634674
8193 0.21571145951747894
8194 0.21550098061561584
8195 0.2152813971042633
8196 0.21507036685943604
8197 0.2148621678352356
8198 0.21464748680591583
8199 0.2144359052181244
8200 0.21421723067760468
8201 0.21401430666446686
8202 0.21381106972694397
8203 0.21359306573867798
8204 0.21337947249412537
8205 0.21316993236541748
8206 0.21296115219593048
8207 0.212748721241951
8208 0.2125357687473297
8209 0.21233248710632324
8210 0.21212930977344513
8211 0.21191620826721191
8212 0.2117040604352951
8213 0.2114984393119812
8214 0.2112933099269867
8215 0.21108223497867584
8216 0.21087303757

8531 0.15498782694339752
8532 0.15483233332633972
8533 0.15468768775463104
8534 0.15453705191612244
8535 0.15438255667686462
8536 0.1542372852563858
8537 0.15408901870250702
8538 0.15394404530525208
8539 0.15379048883914948
8540 0.15364104509353638
8541 0.1534925103187561
8542 0.15333864092826843
8543 0.15319249033927917
8544 0.15304234623908997
8545 0.15289583802223206
8546 0.152747243642807
8547 0.15260441601276398
8548 0.1524527668952942
8549 0.15230512619018555
8550 0.15216079354286194
8551 0.1520104855298996
8552 0.15186621248722076
8553 0.15171608328819275
8554 0.15157273411750793
8555 0.1514255702495575
8556 0.1512763798236847
8557 0.15112918615341187
8558 0.1509835124015808
8559 0.15084093809127808
8560 0.15069857239723206
8561 0.15054981410503387
8562 0.1504005491733551
8563 0.1502586305141449
8564 0.15011316537857056
8565 0.14996951818466187
8566 0.14982493221759796
8567 0.14967457950115204
8568 0.14952819049358368
8569 0.14938637614250183
8570 0.14924731850624084
8571 0.1491

8890 0.10981110483407974
8891 0.10970360785722733
8892 0.1095973327755928
8893 0.1094917580485344
8894 0.10939531028270721
8895 0.10928906500339508
8896 0.10918129980564117
8897 0.1090829074382782
8898 0.10897856205701828
8899 0.10887853056192398
8900 0.10877176374197006
8901 0.10866829752922058
8902 0.108566515147686
8903 0.10846593230962753
8904 0.1083575189113617
8905 0.10825662314891815
8906 0.10815403610467911
8907 0.1080533042550087
8908 0.10795662552118301
8909 0.10785014927387238
8910 0.10774806886911392
8911 0.10764190554618835
8912 0.10753767937421799
8913 0.10743821412324905
8914 0.107339046895504
8915 0.10723507404327393
8916 0.10713718086481094
8917 0.1070338785648346
8918 0.10693075507879257
8919 0.1068304106593132
8920 0.1067313626408577
8921 0.10663267225027084
8922 0.10652634501457214
8923 0.10642481595277786
8924 0.10632575303316116
8925 0.10622779279947281
8926 0.10612469911575317
8927 0.10602236539125443
8928 0.1059233546257019
8929 0.10582268983125687
8930 0.105721

9257 0.07770123332738876
9258 0.07763078063726425
9259 0.07755821943283081
9260 0.0774819478392601
9261 0.07741087675094604
9262 0.0773385688662529
9263 0.07726603746414185
9264 0.07719241082668304
9265 0.07712587714195251
9266 0.07705213129520416
9267 0.07697756588459015
9268 0.07690560817718506
9269 0.07683730125427246
9270 0.0767614096403122
9271 0.07669451832771301
9272 0.07662124931812286
9273 0.07654732465744019
9274 0.07647763937711716
9275 0.07640215754508972
9276 0.07633639872074127
9277 0.07626304030418396
9278 0.07619192451238632
9279 0.07612226158380508
9280 0.07604895532131195
9281 0.07597611099481583
9282 0.07590833306312561
9283 0.07583871483802795
9284 0.07576635479927063
9285 0.07569620758295059
9286 0.07562433928251266
9287 0.07555776089429855
9288 0.07548252493143082
9289 0.07541266083717346
9290 0.07534332573413849
9291 0.07527388632297516
9292 0.07520178705453873
9293 0.0751328244805336
9294 0.07506340742111206
9295 0.07499450445175171
9296 0.07492520660161972
9297

9729 0.05028105154633522
9730 0.050234366208314896
9731 0.05018993467092514
9732 0.05014156550168991
9733 0.05009732395410538
9734 0.05005078390240669
9735 0.050006527453660965
9736 0.04996262118220329
9737 0.04991772398352623
9738 0.04987187683582306
9739 0.04982822388410568
9740 0.049782682210206985
9741 0.049736976623535156
9742 0.04969368875026703
9743 0.04964691027998924
9744 0.049602579325437546
9745 0.049556173384189606
9746 0.04951251298189163
9747 0.049467120319604874
9748 0.04941993206739426
9749 0.04937761276960373
9750 0.04933146759867668
9751 0.04928704723715782
9752 0.049243658781051636
9753 0.049199074506759644
9754 0.04915500804781914
9755 0.04910978674888611
9756 0.04906630888581276
9757 0.04902121052145958
9758 0.04897726699709892
9759 0.048934437334537506
9760 0.04888924956321716
9761 0.04884542524814606
9762 0.048801254481077194
9763 0.048756349831819534
9764 0.0487101711332798
9765 0.0486680343747139
9766 0.04862498492002487
9767 0.04858234152197838
9768 0.04853723